# Jupyter Notebook for YOLOv11 Emotion Detection Model (CPU Test)

## Step 1: Set up the environment

In [ ]:
import os
from ultralytics import YOLO
import yaml

## Step 2: Verify dataset path

In [2]:
dataset_path = r"D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Datasets\YOLOv11s_ED\test-4"
test_img_dir = os.path.join(dataset_path, "train", "images")
test_label_dir = os.path.join(dataset_path, "train", "labels")

In [3]:
# Check if test directories exist
if not os.path.exists(test_img_dir) or not os.path.exists(test_label_dir):
    raise FileNotFoundError(f"Test directories not found at {test_img_dir} or {test_label_dir}")

## Step 3: Update data.yaml with absolute paths

In [4]:
data_yaml_path = os.path.join(dataset_path, "data.yaml")
with open(data_yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

# Use absolute paths for the test set (use train folder as the test set)
data_yaml["train"] = os.path.abspath(os.path.join(dataset_path, "train/images"))
data_yaml["val"] = ""  # No validation set
data_yaml["test"] = os.path.abspath(os.path.join(dataset_path, "train/images"))  # Use all images for testing
data_yaml["nc"] = 7
data_yaml["names"] = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)

In [5]:
# Verify paths exist
if not os.path.exists(data_yaml["test"]):
    raise FileNotFoundError(f"Test image directory not found: {data_yaml['test']}")

## Step 4: Load fine-tuned YOLOv11 model

In [6]:
model_path = r"D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Models\YOLOv11s_Emotion_Detection.pt"

In [7]:
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")
model = YOLO(model_path)

c:\Users\ASUS\miniconda3\envs\yolo11ed_cpu\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Step 5: Batch evaluation and inference

In [8]:
# Perform evaluation (mAP) and inference on the test set
val_results = model.val(
    data=data_yaml_path,
    batch=32,  # Batch size (CPU can handle similar batch size)
    conf=0.25,  # Confidence threshold for inference
    save=True,  # Save inference results
    split="test",  # Use test set (all 536 images)
    device="cpu"  # Force CPU usage
)

# Extract and print mAP metrics
print("Test Metrics:")
print(f"mAP@50: {val_results.box.map50:.4f}")
print(f"mAP@50:95: {val_results.box.map:.4f}")

Ultralytics 8.3.168  Python-3.11.13 torch-2.7.1+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
YOLO11s summary (fused): 100 layers, 9,415,509 parameters, 0 gradients, 21.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 6.60MB/s]

val: Fast image access  (ping: 0.20.1 ms, read: 21.66.8 MB/s, size: 204.1 KB)



val: Scanning D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Datasets\YOLOv11s_ED\test-4\train\labels... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:01<00:00, 378.30it/s]

val: New cache created: D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Datasets\YOLOv11s_ED\test-4\train\labels.cache



c:\Users\ASUS\miniconda3\envs\yolo11ed_cpu\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:42<00:00,  2.53s/it]


                   all        536      19056      0.206      0.215       0.16      0.111
                 Angry        523       1859      0.201      0.114      0.121     0.0819
               Disgust        354        701      0.182      0.173      0.117     0.0892
                  Fear        188        292    0.00932     0.0274    0.00499    0.00146
                 Happy        536       2342      0.228      0.418      0.251      0.177
               Neutral        536      10305      0.546      0.607      0.462      0.305
                   Sad        514       2310      0.158     0.0281     0.0815      0.064
              Surprise        494       1247      0.117      0.134     0.0853     0.0565
Speed: 0.5ms preprocess, 65.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\val
Test Metrics:
mAP@50: 0.1603
mAP@50:95: 0.1106


In [9]:
# Extract speed metrics
preprocess_time = val_results.speed['preprocess']  # ms per image
inference_time = val_results.speed['inference']    # ms per image
postprocess_time = val_results.speed['postprocess']  # ms per image
total_time_per_image = preprocess_time + inference_time + postprocess_time  # ms

# Calculate FPS (per image)
fps = 1000 / total_time_per_image if total_time_per_image > 0 else 0

# Calculate inference time per face, latency per face, and FPS per face
num_images = 536  # Total images in dataset
num_instances = sum(val_results.nt_per_class)  # Total instances
avg_faces_per_image = num_instances / num_images if num_images > 0 else 0
inference_time_per_face = inference_time / avg_faces_per_image if avg_faces_per_image > 0 else 0
latency_per_face = total_time_per_image / avg_faces_per_image if avg_faces_per_image > 0 else 0
fps_per_face = 1000 / inference_time_per_face if inference_time_per_face > 0 else 0

# Print speed metrics, FPS, and per-face metrics
print("\nSpeed Metrics (per image):")
print(f"Preprocess: {preprocess_time:.1f}ms")
print(f"Inference: {inference_time:.1f}ms")
print(f"Postprocess: {postprocess_time:.1f}ms")
print(f"Total (Latency): {total_time_per_image:.1f}ms")
print(f"FPS: {fps:.2f}")
print(f"\nPer Face Metrics:")
print(f"Average faces per image: {avg_faces_per_image:.2f}")
print(f"Inference time per face: {inference_time_per_face:.3f}ms")
print(f"Latency per face: {latency_per_face:.3f}ms")
print(f"FPS per face: {fps_per_face:.2f}")


Speed Metrics (per image):
Preprocess: 0.5ms
Inference: 65.3ms
Postprocess: 0.4ms
Total (Latency): 66.2ms
FPS: 15.11

Per Face Metrics:
Average faces per image: 35.55
Inference time per face: 1.836ms
Latency per face: 1.862ms
FPS per face: 544.80


In [10]:
results_save_path = r"D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Models\YOLOv11s_RT-FER_Test_CPU"
os.makedirs(results_save_path, exist_ok=True)  # Ensure directory exists

In [11]:
# Save metrics to local directory
metrics_file = os.path.join(results_save_path, "test_metrics.txt")
os.makedirs(os.path.dirname(metrics_file), exist_ok=True)  # Ensure parent directory exists
with open(metrics_file, "w") as f:
    f.write(f"Test Metrics:\n")
    f.write(f"mAP@50: {val_results.box.map50:.4f}\n")
    f.write(f"mAP@50:95: {val_results.box.map:.4f}\n")
    f.write(f"\nSpeed Metrics (per image):\n")
    f.write(f"Preprocess: {preprocess_time:.1f}ms\n")
    f.write(f"Inference: {inference_time:.1f}ms\n")
    f.write(f"Postprocess: {postprocess_time:.1f}ms\n")
    f.write(f"Total (Latency): {total_time_per_image:.1f}ms\n")
    f.write(f"FPS: {fps:.2f}\n")
    f.write(f"\nPer Face Metrics:\n")
    f.write(f"Average faces per image: {avg_faces_per_image:.2f}\n")
    f.write(f"Inference time per face: {inference_time_per_face:.3f}ms\n")
    f.write(f"Latency per face: {latency_per_face:.3f}ms\n")
    f.write(f"FPS per face: {fps_per_face:.2f}\n")
print(f"Metrics saved to {metrics_file}")

Metrics saved to D:\College\Semester 8\Skripsi\Program (Google Colab)\Programs\Models\YOLOv11s_RT-FER_Test_CPU\test_metrics.txt
